In [112]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import to_date, date_format

In [113]:
spark = SparkSession.builder.config("spark.jars", "/Drivers/SQL_Sever/jdbc/postgresql-42.7.3.jar")\
.getOrCreate()

In [114]:
table_lst=['goalKeepers','players','matches']
dataframe={}
for table in table_lst:
    df=spark.read.format('jdbc')\
    .option('url','jdbc:postgresql://localhost:5432/Football')\
    .option('driver','org.postgresql.Driver')\
    .option('dbtable',f'{table}')\
    .option('user','root')\
    .option('password','root')\
    .load()
    dataframe[table]=df

In [115]:
print(dataframe.keys())

dict_keys(['goalKeepers', 'players', 'matches'])


In [116]:
matches=dataframe['matches']
players=dataframe['players']
goalkeepers=dataframe['goalKeepers']

In [153]:
print(type(dataframe['matches']))  # Should be <class 'pyspark.sql.dataframe.DataFrame'>

matches = dataframe['matches']
print(type(matches))  # Should be <class 'pyspark.sql.dataframe.DataFrame'>




<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [118]:
matches.createOrReplaceTempView("matches")

In [120]:
distinct_matches = spark.sql ("""
select Count(distinct Date) from matches

""").show()

+--------------------+
|count(DISTINCT Date)|
+--------------------+
|                 214|
+--------------------+



In [121]:
matches.printSchema()

root
 |-- Season: string (nullable = true)
 |-- Tour: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Opponent: string (nullable = true)
 |-- HoAw: string (nullable = true)
 |-- ArsenalScore: string (nullable = true)
 |-- OpponentScore: string (nullable = true)
 |-- Stadium: string (nullable = true)
 |-- Attendance: string (nullable = true)
 |-- Coach: string (nullable = true)
 |-- Referee: string (nullable = true)



#### Making sure that all matches are unique for each date

In [122]:
matches = spark.sql ("""
select count(Date) from matches
""").show()

+-----------+
|count(Date)|
+-----------+
|        214|
+-----------+



#### Know the Max and Min date to create the DimDate Later

In [123]:
Dates=spark.sql("""
    select max(Date) as Max_date,min(Date) as Min_date
    from matches
""").show()

+----------+----------+
|  Max_date|  Min_date|
+----------+----------+
|2023-02-25|2017-08-11|
+----------+----------+



In [155]:
print(type(matches))

<class 'pyspark.sql.dataframe.DataFrame'>


##### Adding MatchID column to Matches DataFrame


In [154]:

DimMatch = matches.withColumn("MatchID", monotonically_increasing_id())
DimMatch.show()


+-------+----+----------+--------+--------------+----+------------+-------------+-----------------+----------+-------------+---------------+-------+
| Season|Tour|      Date|    Time|      Opponent|HoAw|ArsenalScore|OpponentScore|          Stadium|Attendance|        Coach|        Referee|MatchID|
+-------+----+----------+--------+--------------+----+------------+-------------+-----------------+----------+-------------+---------------+-------+
|2017/18|   1|2017-08-11|20:45:00|     Leicester|home|           4|            3| Emirates Stadium|     59387|Arsène Wenger|      Mike Dean|      0|
|2017/18|   2|2017-08-19|18:30:00|    Stoke City|away|           0|            1|   bet365 Stadium|     29459|Arsène Wenger| Andre Marriner|      1|
|2017/18|   3|2017-08-27|17:00:00|     Liverpool|away|           0|            4|          Anfield|     53206|Arsène Wenger|   Craig Pawson|      2|
|2017/18|   4|2017-09-09|16:00:00|   Bournemouth|home|           3|            0| Emirates Stadium|     59

In [156]:
DimMatch.columns

['Season',
 'Tour',
 'Date',
 'Time',
 'Opponent',
 'HoAw',
 'ArsenalScore',
 'OpponentScore',
 'Stadium',
 'Attendance',
 'Coach',
 'Referee',
 'MatchID']

#### Loading the DimMatch for DWH Schema in ArsenalFC Database in Post

In [157]:
DimMatch=DimMatch.withColumn("FormattedDate",date_format(to_date("Date","yyyy-MM-dd"),"yyyy-MM-dd"))

In [158]:
DimMatch.write.format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/Football") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "dwh.DimArsenalMatches") \
    .option("user", "root") \
    .option("password", "root") \
    .mode("overwrite") \
    .save()

### Transforming Players Data & Creating the DimPlayers and Fact Players

In [159]:
players.createOrReplaceTempView("players")

In [160]:
players.columns

['LastName',
 'FirstName',
 'Date',
 'Start',
 'Pos',
 'Min',
 'G',
 'A',
 'PK',
 'PKA',
 'S',
 'SoT',
 'YK',
 'RK',
 'Touches',
 'Tackles',
 'Ints',
 'Blocks',
 'xG',
 'npxG',
 'xAG',
 'Passes',
 'PassesA',
 'PrgPas',
 'Carries',
 'PrgCar',
 'Line',
 'C',
 'fullname']

In [161]:
spark.sql("""
    select concat(FirstName, " ", LastName) as fullname
    from players
""").show()

+-------------------+
|           fullname|
+-------------------+
|    Hector Bellerin|
|     Mohamed Elneny|
|        Rob Holding|
|     Sead Kolasinac|
|Alexandre Lacazette|
|      Nacho Monreal|
|         Mesut Ozil|
|      Danny Welbeck|
|       Granit Xhaka|
|       Aaron Ramsey|
|    Hector Bellerin|
|     Sead Kolasinac|
|Alexandre Lacazette|
|      Nacho Monreal|
|   Shkodran Mustafi|
|         Mesut Ozil|
|       Aaron Ramsey|
|      Danny Welbeck|
|       Granit Xhaka|
|         Alex Iwobi|
+-------------------+
only showing top 20 rows



In [162]:
distinct_players=spark.sql("""
        select distinct concat(FirstName," ",LastName) as FullName
        from players
""")
distinct_players.show(10)

+----------------+
|        FullName|
+----------------+
|Emile Smith Rowe|
| Folarin Balogun|
| Hector Bellerin|
|     Joe Willock|
|  William Saliba|
|    Aaron Ramsey|
|     Bukayo Saka|
|  Kieran Tierney|
|Shkodran Mustafi|
| Daniel Ceballos|
+----------------+
only showing top 10 rows



### Count the distinct Date to check if it match with DimMatches or not

In [163]:
spark.sql("""
    select count(distinct date) as count
    from players
""").show()

+-----+
|count|
+-----+
|  214|
+-----+



In [164]:
distinct_players=distinct_players.withColumn("playerID",monotonically_increasing_id())

### Adding fullname column to Player so we can create the DimPlayers joinig it with Distinct_Player Table

In [165]:
players=players.withColumn('fullname',concat_ws(" ",col('FirstName'),col('LastName')))

In [166]:
players.select('fullname').show(5,False)

+-------------------+
|fullname           |
+-------------------+
|Hector Bellerin    |
|Mohamed Elneny     |
|Rob Holding        |
|Sead Kolasinac     |
|Alexandre Lacazette|
+-------------------+
only showing top 5 rows



In [167]:
players.show(2)

+--------+---------+---------+-----+---+---+---+---+---+---+---+---+---+---+-------+-------+----+------+---+----+---+------+-------+------+-------+------+----------+---+---------------+
|LastName|FirstName|     Date|Start|Pos|Min|  G|  A| PK|PKA|  S|SoT| YK| RK|Touches|Tackles|Ints|Blocks| xG|npxG|xAG|Passes|PassesA|PrgPas|Carries|PrgCar|      Line|  C|       fullname|
+--------+---------+---------+-----+---+---+---+---+---+---+---+---+---+---+-------+-------+----+------+---+----+---+------+-------+------+-------+------+----------+---+---------------+
|Bellerin|   Hector|8/11/2017|    1| WB| 90|  0|  0|  0|  0|  1|  1|  0|  0|     79|      3|   0|     0|0.3| 0.3|  0|    61|     70|     3|     51|     1|  Defender|  0|Hector Bellerin|
|  Elneny|  Mohamed|8/11/2017|    1| CM| 66|  0|  1|  0|  0|  1|  0|  0|  0|     82|      4|   0|     2|  0|   0|0.1|    65|     72|     4|     57|     0|Midfielder|  0| Mohamed Elneny|
+--------+---------+---------+-----+---+---+---+---+---+---+---+---+--

In [168]:
players.columns

['LastName',
 'FirstName',
 'Date',
 'Start',
 'Pos',
 'Min',
 'G',
 'A',
 'PK',
 'PKA',
 'S',
 'SoT',
 'YK',
 'RK',
 'Touches',
 'Tackles',
 'Ints',
 'Blocks',
 'xG',
 'npxG',
 'xAG',
 'Passes',
 'PassesA',
 'PrgPas',
 'Carries',
 'PrgCar',
 'Line',
 'C',
 'fullname']

Creating DimPlayers

In [169]:
DimPlayers=players.join(distinct_players,on='fullname',how='inner')

In [170]:
DimPlayers.columns

['fullname',
 'LastName',
 'FirstName',
 'Date',
 'Start',
 'Pos',
 'Min',
 'G',
 'A',
 'PK',
 'PKA',
 'S',
 'SoT',
 'YK',
 'RK',
 'Touches',
 'Tackles',
 'Ints',
 'Blocks',
 'xG',
 'npxG',
 'xAG',
 'Passes',
 'PassesA',
 'PrgPas',
 'Carries',
 'PrgCar',
 'Line',
 'C',
 'playerID']

In [171]:
DimPlayers=DimPlayers.withColumn('FormattedDate',date_format(to_date('Date','m/d/yyyy'),'yyyy-MM-dd'))

In [172]:
DimPlayers.show(1)

+----------------+----------+---------+----------+-----+---+---+---+---+---+---+---+---+---+---+-------+-------+----+------+---+----+---+------+-------+------+-------+------+----------+---+--------+-------------+
|        fullname|  LastName|FirstName|      Date|Start|Pos|Min|  G|  A| PK|PKA|  S|SoT| YK| RK|Touches|Tackles|Ints|Blocks| xG|npxG|xAG|Passes|PassesA|PrgPas|Carries|PrgCar|      Line|  C|playerID|FormattedDate|
+----------------+----------+---------+----------+-----+---+---+---+---+---+---+---+---+---+---+-------+-------+----+------+---+----+---+------+-------+------+-------+------+----------+---+--------+-------------+
|Emile Smith Rowe|Smith Rowe|    Emile|12/15/2019|    0| AM| 32|  0|  0|  0|  0|  0|  0|  0|  0|     15|      1|   0|     0|  0|   0|  0|    12|     13|     0|     11|     2|Midfielder|  0|       0|   2019-01-15|
+----------------+----------+---------+----------+-----+---+---+---+---+---+---+---+---+---+---+-------+-------+----+------+---+----+---+------+----

### Creating the FactPayers

In [175]:
DimMatch = DimMatch.withColumn("Date", to_date("Date", "yyyy-M-d"))

DimPlayers = DimPlayers.withColumn("Date", to_date("Date", "yyyy-M-d"))


In [176]:
DimPlayers.schema

StructType([StructField('fullname', StringType(), False), StructField('LastName', StringType(), True), StructField('FirstName', StringType(), True), StructField('Date', DateType(), True), StructField('Start', StringType(), True), StructField('Pos', StringType(), True), StructField('Min', StringType(), True), StructField('G', StringType(), True), StructField('A', StringType(), True), StructField('PK', StringType(), True), StructField('PKA', StringType(), True), StructField('S', StringType(), True), StructField('SoT', StringType(), True), StructField('YK', StringType(), True), StructField('RK', StringType(), True), StructField('Touches', StringType(), True), StructField('Tackles', StringType(), True), StructField('Ints', StringType(), True), StructField('Blocks', StringType(), True), StructField('xG', StringType(), True), StructField('npxG', StringType(), True), StructField('xAG', StringType(), True), StructField('Passes', StringType(), True), StructField('PassesA', StringType(), True), 

In [179]:
FactPlayers=DimMatch.join(DimPlayers,on='FormattedDate',how='left')

In [202]:
FactPlayers.show()

+-------------+-------+----+--------+--------------+----+------------+-------------+-----------------+----------+-------------+---------------+-------+--------+--------+---------+-----+----+----+----+----+----+----+----+----+----+----+-------+-------+----+------+----+----+----+------+-------+------+-------+------+----+----+--------+
|FormattedDate| Season|Tour|    Time|      Opponent|HoAw|ArsenalScore|OpponentScore|          Stadium|Attendance|        Coach|        Referee|MatchID|fullname|LastName|FirstName|Start| Pos| Min|   G|   A|  PK| PKA|   S| SoT|  YK|  RK|Touches|Tackles|Ints|Blocks|  xG|npxG| xAG|Passes|PassesA|PrgPas|Carries|PrgCar|Line|   C|playerID|
+-------------+-------+----+--------+--------------+----+------------+-------------+-----------------+----------+-------------+---------------+-------+--------+--------+---------+-----+----+----+----+----+----+----+----+----+----+----+-------+-------+----+------+----+----+----+------+-------+------+-------+------+----+----+-----

In [204]:
FactPlayers = FactPlayers.drop('Date')


In [208]:
spark.sql( "SELECT * FROM fact_players WHERE " + ' OR '.join([f"{col} IS NULL" for col in FactPlayers.columns])
         ).show()


+-------------+-------+----+----------+--------+--------------+----+------------+-------------+-----------------+----------+-----------------+---------------+-------+--------+--------+---------+----+-----+----+----+----+----+----+----+----+----+----+----+-------+-------+----+------+----+----+----+------+-------+------+-------+------+----+----+--------+
|FormattedDate| Season|Tour|      Date|    Time|      Opponent|HoAw|ArsenalScore|OpponentScore|          Stadium|Attendance|            Coach|        Referee|MatchID|fullname|LastName|FirstName|Date|Start| Pos| Min|   G|   A|  PK| PKA|   S| SoT|  YK|  RK|Touches|Tackles|Ints|Blocks|  xG|npxG| xAG|Passes|PassesA|PrgPas|Carries|PrgCar|Line|   C|playerID|
+-------------+-------+----+----------+--------+--------------+----+------------+-------------+-----------------+----------+-----------------+---------------+-------+--------+--------+---------+----+-----+----+----+----+----+----+----+----+----+----+----+-------+-------+----+------+----+--

In [207]:
FactPlayers.columns

['FormattedDate',
 'Season',
 'Tour',
 'Time',
 'Opponent',
 'HoAw',
 'ArsenalScore',
 'OpponentScore',
 'Stadium',
 'Attendance',
 'Coach',
 'Referee',
 'MatchID',
 'fullname',
 'LastName',
 'FirstName',
 'Start',
 'Pos',
 'Min',
 'G',
 'A',
 'PK',
 'PKA',
 'S',
 'SoT',
 'YK',
 'RK',
 'Touches',
 'Tackles',
 'Ints',
 'Blocks',
 'xG',
 'npxG',
 'xAG',
 'Passes',
 'PassesA',
 'PrgPas',
 'Carries',
 'PrgCar',
 'Line',
 'C',
 'playerID']

In [209]:
FactPlayers=FactPlayers.drop('Season',
 'Tour',
 'Time',
 'Opponent',
 'HoAw',
 'Stadium','Coach',
 'Referee',
   'fullname',
 'LastName',
 'FirstName','Line')

In [213]:
FactPlayers.columns

['FormattedDate',
 'ArsenalScore',
 'OpponentScore',
 'Attendance',
 'MatchID',
 'Start',
 'Pos',
 'Min',
 'G',
 'A',
 'PK',
 'PKA',
 'S',
 'SoT',
 'YK',
 'RK',
 'Touches',
 'Tackles',
 'Ints',
 'Blocks',
 'xG',
 'npxG',
 'xAG',
 'Passes',
 'PassesA',
 'PrgPas',
 'Carries',
 'PrgCar',
 'C',
 'playerID']

#### Loading the FactPlayers for DWH Schema in ArsenalFC Database in PostgreSql

In [215]:
FactPlayers.write.format("jdbc") \
    .option("url","jdbc:postgresql://localhost:5432/Football")\
    .option("driver","org.postgresql.Driver") \
    .option("dbtable","dwh.FactArsenalPlayers") \
    .option("user","root") \
    .option("password","root")\
    .mode("overwrite") \
    .save()

In [216]:
FactPlayers.columns

['FormattedDate',
 'ArsenalScore',
 'OpponentScore',
 'Attendance',
 'MatchID',
 'Start',
 'Pos',
 'Min',
 'G',
 'A',
 'PK',
 'PKA',
 'S',
 'SoT',
 'YK',
 'RK',
 'Touches',
 'Tackles',
 'Ints',
 'Blocks',
 'xG',
 'npxG',
 'xAG',
 'Passes',
 'PassesA',
 'PrgPas',
 'Carries',
 'PrgCar',
 'C',
 'playerID']

In [217]:
DimPlayers= DimPlayers.drop('Date',
 'Start',
 'Min',
 'G',
 'A',
 'PK',
 'PKA',
 'S',
 'SoT',
 'YK',
 'RK',
 'Touches',
 'Tackles',
 'Ints',
 'Blocks',
 'xG',
 'npxG',
 'xAG',
 'Passes',
 'PassesA',
 'PrgPas',
 'Carries',
 'PrgCar',
 'Line',
 'C','FormattedDate', 'Pos')



In [218]:
DimPlayers.createOrReplaceTempView("DimPlayers")

In [219]:
DimPlayers.columns

['fullname', 'LastName', 'FirstName', 'playerID']

In [220]:
DimPlayers.show()

+----------------+----------+---------+--------+
|        fullname|  LastName|FirstName|playerID|
+----------------+----------+---------+--------+
|Emile Smith Rowe|Smith Rowe|    Emile|       0|
|Emile Smith Rowe|Smith Rowe|    Emile|       0|
|Emile Smith Rowe|Smith Rowe|    Emile|       0|
|Emile Smith Rowe|Smith Rowe|    Emile|       0|
|Emile Smith Rowe|Smith Rowe|    Emile|       0|
|Emile Smith Rowe|Smith Rowe|    Emile|       0|
|Emile Smith Rowe|Smith Rowe|    Emile|       0|
|Emile Smith Rowe|Smith Rowe|    Emile|       0|
|Emile Smith Rowe|Smith Rowe|    Emile|       0|
|Emile Smith Rowe|Smith Rowe|    Emile|       0|
|Emile Smith Rowe|Smith Rowe|    Emile|       0|
|Emile Smith Rowe|Smith Rowe|    Emile|       0|
|Emile Smith Rowe|Smith Rowe|    Emile|       0|
|Emile Smith Rowe|Smith Rowe|    Emile|       0|
|Emile Smith Rowe|Smith Rowe|    Emile|       0|
|Emile Smith Rowe|Smith Rowe|    Emile|       0|
|Emile Smith Rowe|Smith Rowe|    Emile|       0|
|Emile Smith Rowe|Sm

In [222]:
DimPlayers.write.format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/Football") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "dwh.DimArsenalPlayers") \
    .option("user", "root") \
    .option("password", "root") \
    .mode("overwrite") \
    .save()
